In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('merged_cpi_lego.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11744 entries, 0 to 11743
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        11744 non-null  int64  
 1   Number            11744 non-null  object 
 2   Theme             11744 non-null  object 
 3   Subtheme          11744 non-null  object 
 4   Year              11744 non-null  int64  
 5   Set name          11744 non-null  object 
 6   Pieces            11744 non-null  float64
 7   RRP (USD)         11744 non-null  float64
 8   Value new (USD)   11744 non-null  float64
 9   Value used (USD)  11744 non-null  float64
 10  CPI Mean          11744 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1009.4+ KB


In [3]:
df.drop(columns=['Unnamed: 0','Number'], inplace=True)


### Dummy Features

In [4]:
dummy = pd.get_dummies(df[['Theme']])
df = pd.concat([df, dummy], axis=1)
df.drop(columns=['Theme','Subtheme','Set name'], inplace=True)
dummy.head()


,Theme_4 Juniors,Theme_Action Wheelers,Theme_Advanced models,Theme_Adventurers,Theme_Agents,Theme_Alpha Team,Theme_Aqua Raiders,Theme_Aquazone,Theme_Architecture,Theme_Art,...,Theme_Ultra Agents,Theme_Unikitty,Theme_Universal Building Set,Theme_Vidiyo,Theme_Vikings,Theme_Western,Theme_World City,Theme_World Racers,Theme_Xtra,Theme_Znap
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
for col in df.columns[:3]:  # Selecting the first 3 columns
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = (df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))
    df = df[~outliers]

### Train Test Splt ###

In [6]:
X = df.drop(columns=['Value new (USD)','RRP (USD)','Value used (USD)'])
y = df[['Value new (USD)','RRP (USD)','Value used (USD)']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train)

       Year  Pieces    CPI Mean  Theme_4 Juniors  Theme_Action Wheelers  \
1429   2000    15.0  312.800000            False                  False   
4569   2010   133.0  355.897385            False                  False   
393    1991   193.0  278.869231            False                  False   
6362   2014     6.0  373.448769            False                  False   
11530  2023   103.0  436.901846            False                  False   
...     ...     ...         ...              ...                    ...   
8146   2017   110.0  381.418077            False                  False   
5189   2012     7.0  366.702077            False                  False   
2668   2004   170.0  328.507692            False                  False   
10276  2021   110.0  405.587462            False                  False   
9348   2019   110.0  391.299154            False                  False   

       Theme_Advanced models  Theme_Adventurers  Theme_Agents  \
1429                   False      

### Standarize Data

In [7]:
# scaler = preprocessing.StandardScaler()
# features = X_train.columns
# # scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [8]:
train_data = pd.concat([pd.DataFrame(X_train), y_train], axis = 1)
test_data = pd.concat([pd.DataFrame(X_test), y_test], axis = 1)
train_data.to_csv('train_data.csv', index = False)
test_data.to_csv('test_data.csv', index = False)